In [ ]:
import param
import quest
import panel as pn
import holoviews as hv
import geoviews as gv
import xarray as xr
import cartopy.crs as ccrs

from holoviews.operation.datashader import rasterize

hv.extension('bokeh')

## Select tiles to download

In [ ]:
quest_service = 'svc://wmts:seamless_imagery'
tile_service_options = quest.api.download_options(quest_service, fmt='param')[quest_service]
tile_service_options.params()['bbox'].precedence = -1  # hide bbox input
pn.panel(tile_service_options.param)

In [ ]:
tiles = gv.WMTS(tile_service_options.url).options(width=950, height=600, global_extent=True)
boxes = gv.Polygons([]).options(fill_alpha=0.4, color='blue', line_color='blue', line_width=2)
box_stream = hv.streams.BoxEdit(source=boxes, num_objects=1)
tiles * boxes

## Define query and download

In [ ]:
if box_stream.element:
    data = box_stream.data
    bbox = [data['x0'][0], data['y0'][0], data['x1'][0], data['y1'][0]]
else:
    bbox = [-72.43925984610391, 45.8471360126193, -68.81252476472281, 47.856449699679516]
tile_service_options.bbox = bbox
print(bbox)

In [ ]:
basemap_features = quest.api.get_features(quest_service)
collection_name = 'examples'
if collection_name in quest.api.get_collections():
    pass
else:
    quest.api.new_collection(collection_name)

collection_feature = quest.api.add_features(collection_name, basemap_features[0])
options = dict(tile_service_options.param.get_param_values())
staged_id = quest.api.stage_for_download(collection_feature, options=options)[0]
quest.api.download_datasets(staged_id, raise_on_error=True)
meta = quest.api.get_metadata(staged_id)[staged_id]
meta

## Open and display downloaded tiles

In [ ]:
meta = quest.api.get_metadata(staged_id)
path = list(meta.values())[0]['file_path']
rgb = gv.project(gv.RGB.load_tiff(path, crs=ccrs.PlateCarree()))
gv.tile_sources.Wikipedia * rasterize(rgb).options(width=800, height=600)